In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from google.colab import files
uploaded = files.upload()


Sigmoid computes the sigmoid activation function of Z, giving us A.

In [ ]:
def Sigmoid(z):
  return (1 / (1 + np.exp(-z.astype(float))))

 The main function calls the Train function to get the final values of the weights and bias. It then calculates the success rate of the model.

In [ ]:
def main():
    # yourfile represents your csv file name
    W,B = Train('yourfile.csv',2,0.001)
    print("Weights:\n",W)
    print("Bias: ",B)
    successRate = Test('yourfile.csv',W,B)
    print("Success Rate: ",np.sum(successRate))

The Save Result function adds a new column to the test results file that represents the predictions. Everything is saved in a new file not to disturb the program the next time it runs.

In [ ]:
def SaveResult(file,result):
    df = pd.read_csv(file)
    df['Prediction'] = [int(i) for i in np.transpose(result)]
    df.to_csv('test-result.csv')

The Train function will read the file and make a call to the initialize function after transposing both X and Y. It will then make a call to the Gradient Descent function that will return the final W and B. 

In [ ]:
def Train(file, epochs, alpha):
    X,Y = Read(file)
    X = np.transpose(X)
    
    Y = np.transpose(Y)
    nx,m = np.shape(X)
    W,B = Initialize(nx)
    W,B = GradientDescent(W,B,X,Y,epochs, alpha)
    
    return W,B

The Test function will execute the gradient descent for one epoch. It will then return the success rate that will be displayed at the end.

In [ ]:
def Test(file,W,B):
    X,Y = Read(file)
    X = np.transpose(X)
    # X = Normalize(X)
    Y = np.transpose(Y)
    Z = np.dot(W.T, X) + B
    A = Sigmoid(Z)
    A = np.round(A)
    diff = np.sum(np.abs(A-Y))
    successRate = (1 - (diff/np.size(A)))/X.shape[1]
    return successRate

The Read function reads data from the file and manages both the splitting and cleaning of the data.

In [ ]:
def Read(file):
    data = pd.read_csv(file, delimiter=';')
    data_train, data_test = splitting(data)
    
    Y_train = data_train.filter(['y'], axis=1)
    X_train = data_train.drop(['y'], axis=1)
    X_train = Clean_X(X_train)
    Y_train = Clean_Y(Y_train)

    Y_test = data_test.filter(['y'], axis=1)
    X_test = data_test.drop(['y'], axis=1)
    X_test = Clean_X(X_test)
    Y_test = Clean_Y(Y_test)

    # print(X)
    # print(Y)
    return X_train, Y_train

This function initializes the weights with random values and the bias with zero.

In [ ]:
def Initialize(nx):
    W = np.random.randn(nx, 1)* 0.01
    B = 0
    return W,B

The Gradient Descent function follows the algorithm seen in class. It uses the forward then backward propagation with a calculation of the cost and an update of the weights and bias.

In [ ]:
def GradientDescent(W,B,X,Y,epochs,learnRate):
    m,nx = np.shape(X)
    costs = []
    print("Working with learn rate",learnRate,"and",epochs,"iterations")
    for i in range (epochs):
        print("Epoch: ", i+1)
        Z = np.array(np.dot(W.T,X) + B, dtype=np.float32)
        A = Sigmoid(Z.astype(float))
        cost = 1/m * - np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))
        print("Cost: ", np.sum(cost))
        if i==0 or i==epochs-1:
            costs.append(cost)
        dz = A-Y
        dw = (1/m) * np.dot(X,dz.T)
        db = (1/m) * np.sum(dz)
        W = W - learnRate*dw
        B = B - learnRate*np.mean(db)
        print("Accuracy: ", accuracy(classify(A), Y))
    print("Cost started with:",np.sum(costs[0]),"and ended with:",np.sum(costs[1]))
    return W,B

In [ ]:
 main()

Working with learn rate 0.001 and 2 iterations
Epoch:  1
Cost:  1575.4242797371958
Accuracy:  0.19929219199292192
Epoch:  2
Cost:  950.1992202986967
Accuracy:  0.8835434638354347
Cost started with: 1575.4242797371958 and ended with: 950.1992202986967
Weights:
 [[-0.3525409111184808]
 [-0.35296488472956805]
 [-0.3342792543247544]
 [-0.31804335283437896]
 [-0.013247542651524155]
 [-0.11877234023189051]
 [-0.722764998092892]
 [-0.22812114820160095]
 [-0.33410302220469607]
 [-0.5606555824644339]
 [-0.3334854483450311]
 [-0.05715184773972089]
 [-0.040543445136744315]
 [-0.0253435084518776]
 [-0.009362350480822764]
 [-0.3386486471927609]]
Bias:  -3.3243997317697746e-05
Success Rate:  0.9999968259895168


The Splitting function splits the data into a training and testing set.

In [ ]:
def splitting(data):
  train, test = train_test_split(data, test_size=0.2)
  return train, test;

The Clean_X function helps cleaning the data into different categories (int, category, date).

In [ ]:
def Clean_X(data):
  data.default.replace(('yes', 'no'), (1, 0), inplace=True)
  data.housing.replace(('yes', 'no'), (1, 0), inplace=True)
  data.loan.replace(('yes', 'no'), (1, 0), inplace=True)
  data.age = (data.age - min(data.age)) / (max(data.age) - min(data.age))
  data.balance = (data.balance - min(data.balance)) / (max(data.balance) - min(data.age))
  data.day = (data.day - min(data.day)) / (max(data.day)- min(data.age))
  data.duration = (data.duration - min(data.duration)) / (max(data.duration)- min(data.age))
  data.campaign = (data.campaign - min(data.campaign)) / (max(data.campaign)- min(data.age))
  data.pdays = (data.pdays - min(data.pdays)) / (max(data.pdays)- min(data.age))
  data.previous = (data.previous - min(data.previous)) / (max(data.previous)- min(data.age))

  data.job = pd.get_dummies(data, columns=["job"])
  data.marital = pd.get_dummies(data, columns=["marital"])
  data.education = pd.get_dummies(data, columns=["education"])
  data.contact = pd.get_dummies(data, columns=["contact"])
  data.poutcome = pd.get_dummies(data, columns=["poutcome"])
  data.month = pd.get_dummies(data, columns=["month"])

  return data

The Clean_Y performs the same actions as Clean_X on y only.

In [ ]:
def Clean_Y(data):
  data.y.replace(('yes', 'no'), (1, 0), inplace=True)
  return data

The Accuracy function calculates the accuracy of the model at each iteration.

In [ ]:
def accuracy(predicted, actual):
  diff = predicted - actual
  return 1.0 - (float(np.count_nonzero(diff)) / predicted.shape[0])

The Decision Boundary function returns 0 or 1 according to the probability gotten.

In [ ]:
def decision_boundary(prob):
  return 1 if prob >= .5 else 0

The Classify function will help in classifying the weights into 0s and 1s using the Decision Boundary Function.

In [ ]:
def classify(predictions):
  fun = np.vectorize(decision_boundary)
  return fun(predictions).flatten()